# Framework Comparison

- Numpy vs Tensorflow vs PyTorch
- output = x * y + z


In [4]:
import numpy as np
from datetime import datetime
start = datetime.now()

np.random.seed(0)

N,D = 3,4

x = np.random.randn(N,D)
y = np.random.randn(N,D)
z = np.random.randn(N,D)

a = x * y
b = a + z
c = np.sum(b)

grad_c = 1.0
grad_b = grad_c * np.ones((N,D))
grad_a = grad_b.copy()
grad_z = grad_b.copy()
grad_y = grad_a * y
grad_x = grad_a * x

print(grad_x)
print(grad_y)
print(grad_z)
print(datetime.now()-start)

[[ 1.76405235  0.40015721  0.97873798  2.2408932 ]
 [ 1.86755799 -0.97727788  0.95008842 -0.15135721]
 [-0.10321885  0.4105985   0.14404357  1.45427351]]
[[ 0.76103773  0.12167502  0.44386323  0.33367433]
 [ 1.49407907 -0.20515826  0.3130677  -0.85409574]
 [-2.55298982  0.6536186   0.8644362  -0.74216502]]
[[ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]]
0:00:00.004966


In [5]:
import tensorflow as tf
import numpy as np
from datetime import datetime
start = datetime.now()

with tf.device('/gpu:0'):
    x = tf.placeholder(tf.float32)
    y = tf.placeholder(tf.float32)
    z = tf.placeholder(tf.float32)

    a = x * y
    b = a + z
    c = tf.reduce_sum(b)
    
grad_x, grad_y, grad_z = tf.gradients(c,[x,y,z])

with tf.Session() as sess:
    values = {
        x: np.random.randn(N,D),
        y: np.random.randn(N,D),
        z: np.random.randn(N,D)           
    }
    out = sess.run([c,grad_x,grad_y,grad_z],feed_dict = values)
    c_val, grad_x_val, grad_y_val, grad_z_val = out

print(grad_x_val)
print(grad_y_val)
print(grad_z_val)
print(datetime.now()-start)

[[-1.6138978  -0.21274029 -0.89546657  0.38690251]
 [-0.51080513 -1.18063223 -0.02818223  0.42833188]
 [ 0.06651722  0.30247191 -0.63432211 -0.36274117]]
[[ 1.23029065  1.20237982 -0.38732681 -0.30230275]
 [-1.04855299 -1.42001796 -1.70627022  1.95077538]
 [-0.5096522  -0.43807429 -1.25279534  0.77749038]]
[[ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]
 [ 1.  1.  1.  1.]]
0:00:00.045955


In [6]:
import torch
from torch.autograd import Variable
from datetime import datetime
start = datetime.now()

N,D = 3,4

x = Variable(torch.randn(N,D).cuda(),requires_grad=True)
y = Variable(torch.randn(N,D).cuda(),requires_grad=True)
z = Variable(torch.randn(N,D).cuda(),requires_grad=True)

a = x * y
b = a + z
c = torch.sum(b)

c.backward(gradient=torch.cuda.FloatTensor([1.0]))

print(x.grad)
print(y.grad)
print(z.grad)
print(datetime.now()-start)

Variable containing:
 1.1577 -1.2792 -0.0138 -0.7830
-0.0819  1.0152 -0.3593 -0.3371
 0.2737 -1.0427  0.1550  2.5901
[torch.cuda.FloatTensor of size 3x4 (GPU 0)]

Variable containing:
 0.8626  0.7682  0.4195 -0.4024
-0.9882 -1.3758 -1.8754  0.5969
 0.6585  0.0706 -0.1586  0.0468
[torch.cuda.FloatTensor of size 3x4 (GPU 0)]

Variable containing:
 1  1  1  1
 1  1  1  1
 1  1  1  1
[torch.cuda.FloatTensor of size 3x4 (GPU 0)]

0:00:00.007252
